# Tech Intro

## Main tables

In [8]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [9]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = real_estate_list[8] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [10]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1106 21:40:03.946552 55868 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:34678
I1106 21:40:03.947083 58299 sched.cpp:241] Version: 1.11.0
I1106 21:40:03.947212 56027 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1106 21:40:03.947248 56027 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1106 21:40:03.948469 56029 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-53196


## Read the spec-file from Hadoop and show some data sample

In [11]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+---+----------------------+------------------+--------+------+---------+-------+
|id |name                  |platform_system_id|platform|city  |filial_id|deleted|
+---+----------------------+------------------+--------+------+---------+-------+
|1  |Авиамоторная          |1                 |Cian    |Москва|null     |0      |
|10 |Бабушкинская          |10                |Cian    |Москва|null     |0      |
|100|Преображенская площадь|100               |Cian    |Москва|null     |0      |
|101|Пролетарская          |101               |Cian    |Москва|null     |0      |
|102|Проспект Вернадского  |102               |Cian    |Москва|null     |0      |
|103|Проспект Мира         |103               |Cian    |Москва|null     |0      |
|104|Профсоюзная           |104               |Cian    |Москва|null     |0      |
|105|Пушкинская            |105               |Cian    |Москва|null     |0      |
|106|Речной вокзал         |106               |Cian    |Москва|null     |0      |
|107|Рижская    

## Show columns' properties

In [12]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- platform_system_id: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- city: string (nullable = true)
 |-- filial_id: string (nullable = true)
 |-- deleted: string (nullable = true)



## Total rows (count)

In [13]:
eda_df.count()

431

## Simple func to execute and show the result of the SQL-instruction

In [14]:
def execute(select_instruction: str = "select count(*) from realEstate") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from realEstate"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show()

## Create the temp-table-view from the DF (called as "eda_sql")

In [15]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [16]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|     431|
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [17]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,name,platform_system_id,platform,city,filial_id,deleted
0,0,0,3,0,0,431,0


In [18]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,name,platform_system_id,platform,city,filial_id,deleted
0,0,0,0,0,0,0,430


In [19]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,name,platform_system_id,platform,city,filial_id,deleted
0,0,0,0,0,0,0,0


In [20]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
name,0,0,0
platform_system_id,3,0,0
platform,0,0,0
city,0,0,0
filial_id,431,0,0
deleted,0,430,0


In [23]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [24]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.0
name,0.000,0.000,0.0
platform_system_id,0.696,0.000,0.0
platform,0.000,0.000,0.0
city,0.000,0.000,0.0
filial_id,100.000,0.000,0.0
deleted,0.000,99.768,0.0


### SQL-instructions

In [25]:
#Find dupes or misunderstanding or useless static values 18 845 total
for col in eda_df.columns: 
    execute("select count(distinct("+ col + ")) from eda_sql")
    
        
   




+------------------+
|count(DISTINCT id)|
+------------------+
|               431|
+------------------+

+--------------------+
|count(DISTINCT name)|
+--------------------+
|                 401|
+--------------------+

+----------------------------------+
|count(DISTINCT platform_system_id)|
+----------------------------------+
|                               428|
+----------------------------------+

+------------------------+
|count(DISTINCT platform)|
+------------------------+
|                       1|
+------------------------+

+--------------------+
|count(DISTINCT city)|
+--------------------+
|                   8|
+--------------------+

+-------------------------+
|count(DISTINCT filial_id)|
+-------------------------+
|                        0|
+-------------------------+

+-----------------------+
|count(DISTINCT deleted)|
+-----------------------+
|                      2|
+-----------------------+



## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [26]:
spark.stop()

I1106 21:47:16.411927 55868 sched.cpp:2174] Asked to stop the driver
I1106 21:47:16.412070 56024 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-53196
